# 🧱 Materials Science RAG Platform
## CIF Generation • Property Prediction • Safety-Enforced Synthesis
### Powered by Qwen2.5-7B, Qdrant, and Materials ML Models

---

**This notebook:**
- Runs entirely in Google Colab
- Uses A100 GPU if available
- Implements real models (no mocks)
- Enforces mandatory safety protocols
- Launches Streamlit UI

**DO NOT modify the pipeline logic.** Both this notebook and Streamlit call the same backend.

---

## 🔧 Setup: Environment Detection

In [ ]:
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

print("="*80)
print("ENVIRONMENT DETECTION")
print("="*80)
print(f"Running in Colab: {IN_COLAB}")
print(f"Python version: {sys.version}")

# Note: Will check GPU after installing dependencies
if IN_COLAB:
    print("⚠ GPU detection will be available after installing PyTorch")
else:
    print("Running locally")

print("="*80)

## 📦 Installation

In [ ]:
# ============================================================================
# DEPENDENCY INSTALLATION - COLAB OPTIMIZED
# ============================================================================
# Using Qwen2.5-7B-Instruct with stable 4-bit quantization

print("Step 1: Removing conflicting packages...")
!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers huggingface-hub accelerate bitsandbytes tokenizers peft dgl -q 2>/dev/null

print("\nStep 2: Installing PyTorch with CUDA 12.1 (Colab default)...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q

print("\nStep 3: Installing HuggingFace ecosystem (EXACT versions)...")
!pip install huggingface-hub==0.24.0 --no-deps -q
!pip install tokenizers==0.19.1 --no-deps -q
!pip install transformers==4.43.2 --no-deps -q
!pip install sentence-transformers==2.5.1 --no-deps -q

print("\nStep 4: Installing missing dependencies for HuggingFace packages...")
!pip install -q filelock fsspec packaging pyyaml regex requests tqdm typing-extensions

print("\nStep 5: Installing quantization packages (stable versions)...")
!pip install -q accelerate==0.25.0 bitsandbytes==0.42.0

print("\nStep 6: Installing DGL with CUDA 12.1 support...")
!pip install dgl -f https://data.dgl.ai/wheels/torch-2.5/cu121/repo.html -q

print("\nStep 7: Installing remaining dependencies...")
!pip install -q qdrant-client scikit-learn scipy pillow safetensors

print("\nStep 8: Installing chemistry packages...")
!pip install -q matgl pymatgen ase

print("\n" + "="*70)
print("✅ Installation complete!")
print("="*70)
print("\n⚠️  CRITICAL: Runtime → Restart runtime NOW!")
print("   Then continue from the HuggingFace login cell")
print("="*70)

# Verify critical versions
print("\nInstalled versions:")
import importlib.metadata as metadata
for pkg in ['torch', 'tokenizers', 'huggingface-hub', 'transformers', 'sentence-transformers', 'accelerate', 'bitsandbytes', 'dgl']:
    try:
        print(f"  {pkg}: {metadata.version(pkg)}")
    except:
        print(f"  {pkg}: NOT FOUND")

In [ ]:
# ============================================================================
# POST-RESTART VERIFICATION
# ============================================================================
# Run this cell AFTER restarting the runtime to verify all imports work

import torch
print(f"✓ PyTorch: {torch.__version__}")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  CUDA version: {torch.version.cuda}")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

import transformers
from transformers import PreTrainedModel, AutoTokenizer, AutoModelForCausalLM
print(f"✓ Transformers: {transformers.__version__}")
print(f"  PreTrainedModel: {PreTrainedModel}")

import sentence_transformers
from sentence_transformers import SentenceTransformer
print(f"✓ Sentence Transformers: {sentence_transformers.__version__}")

from qdrant_client import QdrantClient
print(f"✓ Qdrant Client: Available")

try:
    import dgl
    print(f"✓ DGL: {dgl.__version__}")
    print(f"  Backend: {dgl.backend.backend_name}")
except Exception as e:
    print(f"⚠ DGL: Not available - {str(e)[:50]}")
    print("  (MatGL predictions will be skipped)")

try:
    import pymatgen
    print(f"✓ PyMatGen: {pymatgen.__version__}")
except Exception as e:
    print(f"⚠ PyMatGen: Not available - {str(e)[:50]}")

print("\n" + "="*70)
print("✅ All critical dependencies verified!")

print("="*70)print("\nYou can now proceed to upload your project files.")

## 🔑 HuggingFace Login (Bypass Rate Limits)

**Required for Qwen2.5 access:**
1. Create a free account at [huggingface.co](https://huggingface.co)
2. Get your access token from [Settings → Access Tokens](https://huggingface.co/settings/tokens)
3. Accept the Qwen2.5 license at [Qwen/Qwen2.5-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct)
4. Run the cell below and paste your token when prompted

In [ ]:
from huggingface_hub import login

print("="*70)
print("HUGGINGFACE AUTHENTICATION")
print("="*70)
print("\nEnter your HuggingFace access token below.")
print("(Token will not be displayed for security)")
print("-"*70)

try:
    login()
    print("\n✓ Successfully logged in to HuggingFace!")
    print("✓ You can now download Qwen2.5 without rate limits")
except Exception as e:
    print(f"\n✗ Login failed: {e}")
    print("\nTroubleshooting:")
    print("  1. Ensure you created a token at huggingface.co/settings/tokens")
    print("  2. Verify you accepted the Qwen2.5 license")
    print("  3. Check that the token has 'read' permissions")
    
print("="*70)

## 📁 Clone/Setup Project Structure

In [ ]:
# Upload colab_project.zip created by running ./create_colab_zip.sh
from google.colab import files
import zipfile
import os

print("="*80)
print("PROJECT FILE UPLOAD")
print("="*80)
print("\nUpload the colab_project.zip file")
print("(Created by running ./create_colab_zip.sh on your computer)")
print("-" * 80)

# Upload the zip file
uploaded = files.upload()

# Extract and verify
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✓ Extracted to {os.getcwd()}")
        
        # Verify extraction
        print("\n✓ Extracted folders:")
        required_folders = ['pipeline', 'ingestion', 'rag', 'crystal', 'prediction', 'synthesis']
        for folder in required_folders:
            if os.path.exists(folder) and os.path.isdir(folder):
                file_count = len([f for f in os.listdir(folder) if f.endswith('.py')])
                print(f"  ✓ {folder}/ ({file_count} Python files)")
            else:
                print(f"  ✗ {folder}/ - MISSING")
        
        # Check for reaction.csv
        if os.path.exists('reaction.csv'):
            import csv
            with open('reaction.csv', 'r') as f:
                material_count = sum(1 for _ in csv.DictReader(f))
            print(f"  ✓ reaction.csv ({material_count} materials)")
        else:
            print(f"  ✗ reaction.csv - MISSING")
        
        print("\n" + "="*80)
        print("✓ Upload complete! Ready to run pipeline.")
        print("="*80)

In [ ]:
# Verify all files are in place
import os
import sys

print("="*80)
print("VERIFYING PROJECT STRUCTURE")
print("="*80)

# Check current directory
print(f"\nCurrent directory: {os.getcwd()}")

# List all folders
print("\nFolders in current directory:")
for item in sorted(os.listdir('.')):
    if os.path.isdir(item) and not item.startswith('.'):
        print(f"  📁 {item}/")

# Verify required files
print("\nVerifying required files:")
required_files = [
    'pipeline/run_pipeline.py',
    'ingestion/parse_reactions.py',
    'ingestion/precursor_extraction.py',
    'rag/retriever.py',
    'rag/llama_agent.py',
    'crystal/composition_editing.py',
    'crystal/cif_generation.py',
    'prediction/alignff_predict.py',
    'synthesis/hazard_detection.py',
    'synthesis/synthesis_generator.py',
    'reaction.csv'
]

missing = []
for filepath in required_files:
    if os.path.exists(filepath):
        print(f"  ✓ {filepath}")
    else:
        print(f"  ✗ {filepath} - MISSING")
        missing.append(filepath)

# Add to Python path
sys.path.insert(0, os.getcwd())
print(f"\n✓ Added {os.getcwd()} to Python path")

if missing:
    print(f"\n⚠ WARNING: {len(missing)} files missing!")
    print("Please re-upload the zip file or upload folders manually.")
else:
    print("\n" + "="*80)
    print("✓ ALL FILES VERIFIED - Ready to proceed!")
    print("="*80)

## 🚀 Initialize Pipeline (THIS IS THE ONLY SOURCE OF TRUTH)

In [ ]:
# Import the SINGLE SHARED PIPELINE
from pipeline.run_pipeline import MaterialsPipeline
import shutil

# Initialize pipeline with appropriate settings
print("\n🚀 INITIALIZING MATERIALS PIPELINE\n")

# Clean up any locked Qdrant storage from previous runs
qdrant_path = "./qdrant_storage"
if os.path.exists(qdrant_path):
    lock_file = os.path.join(qdrant_path, ".lock")
    if os.path.exists(lock_file):
        print("⚠ Removing stale Qdrant lock file...")
        try:
            os.remove(lock_file)
            print("✓ Lock file removed")
        except Exception as e:
            print(f"⚠ Could not remove lock, recreating storage: {e}")
            shutil.rmtree(qdrant_path)
            print("✓ Storage recreated")

# Use 4-bit quantization if GPU available
use_quantization = torch.cuda.is_available()

# Using Qwen2.5-7B-Instruct (best chemistry knowledge, 128k context)
# Alternative models (all work well with 4-bit quantization):
# - "Qwen/Qwen2.5-7B-Instruct" (Default - 128k context, superior technical knowledge)
# - "mistralai/Mistral-7B-Instruct-v0.3" (32k context, excellent science)
# - "microsoft/Phi-3-medium-4k-instruct" (4k context, best for quantization stability)

pipeline = MaterialsPipeline(
    llama_model_name="Qwen/Qwen2.5-7B-Instruct",
    qdrant_path=qdrant_path,
    embedding_model="all-MiniLM-L6-v2",
    use_4bit=use_quantization
)

print("\n✓ PIPELINE INITIALIZED AND READY")

# Check if database was populated during initialization
print("\n" + "="*80)
print("CHECKING VECTOR DATABASE STATUS")
print("="*80)
pipeline.check_database_status()

## 📊 Load Sample Data

## 📚 Populate Vector Database (Run if Empty)

If the database is empty (0 papers), run this cell to scrape papers from PubMed/arXiv for all materials in reaction.csv. This will take 5-10 minutes.

In [ ]:
# ONLY RUN THIS IF DATABASE IS EMPTY (0 papers)
# This will scrape real papers from PubMed and arXiv for all 42 materials
# Expected time: 5-10 minutes due to API rate limits

print("Starting manual database population...")
print("This will scrape papers for all materials in reaction.csv")
print("Progress will be shown below:\n")

paper_count = pipeline.populate_database_from_reactions(force_reload=False)

if paper_count > 0:
    print(f"\n✓ Database successfully populated with {paper_count} papers!")
    print(f"✓ Literature retrieval is now enabled")
    print(f"✓ Synthesis protocols will include real research data")
else:
    print(f"\n✗ Database population failed")
    print(f"Check error messages above for details")

In [ ]:
import pandas as pd

# Load reactions data (from root directory, not data/)
reactions_df = pd.read_csv('reaction.csv')

print("Sample Materials in Database:")
print(reactions_df[['composition', 'precursors']].head(10).to_string(index=False))
print(f"\nTotal: {len(reactions_df)} materials")
print(f"\nColumns available: {list(reactions_df.columns)}")

## 🔬 Example 1: Basic Material Synthesis

In [ ]:
# Run pipeline for BaTiO3
result = pipeline.run_materials_pipeline(
    composition="BaTiO3",
    substitutions=None,
    generate_cif=True,
    predict_properties=True,
    generate_synthesis=True,
    scrape_papers=False,  # Set True to scrape new papers (slow)
    retrieve_top_k=5
)

print("\n" + "="*80)
print("PIPELINE RESULT")
print("="*80)
print(f"Success: {result.success}")
print(f"Formula: {result.final_formula}")
print(f"Precursors: {', '.join(result.precursors)}")

### Display CIF File

In [ ]:
if result.cif_content:
    print("="*80)
    print("GENERATED CIF FILE")
    print("="*80)
    print(result.cif_content)
    
    # Save to file
    with open(f"{result.final_formula}_generated.cif", 'w') as f:
        f.write(result.cif_content)
    print(f"\n✓ Saved to {result.final_formula}_generated.cif")
else:
    print("⚠ No CIF generated")

### Display Predicted Properties

In [ ]:
if result.predicted_properties:
    print("="*80)
    print(f"PREDICTED PROPERTIES ({result.property_method})")
    print("="*80)
    
    for prop, value in result.predicted_properties.items():
        print(f"{prop:40s}: {value}")
else:
    print("⚠ No properties predicted")

### Display Synthesis Protocol with MANDATORY Safety

In [ ]:
# Run pipeline for Ba2Cl8Ni1Pb1 (from reaction.csv)
result1 = pipeline.run_materials_pipeline(
    composition="Ba2Cl8Ni1Pb1",
    generate_cif=True,
    predict_properties=True,
    generate_synthesis=True,
    retrieve_top_k=5
)

print("\n" + "="*80)
print("PIPELINE RESULT")
print("="*80)
print(f"Formula: {result1.final_formula}")
print(f"Success: {result1.success}")
print(f"CIF Generated: {result1.cif_content is not None}")
print(f"Properties: {result1.predicted_properties is not None}")
print(f"Synthesis: {result1.synthesis_protocol is not None}")

if result1.errors:
    print("\nErrors:")
    for e in result1.errors:
        print(f"  ✗ {e}")

if result1.warnings:
    print("\nWarnings:")
    for w in result1.warnings:
        print(f"  ⚠ {w}")

# Display full synthesis protocol
if result1.synthesis_protocol:
    print("\n" + "="*80)
    print("FULL SYNTHESIS PROTOCOL")
    print("="*80)
    print(result1.synthesis_protocol)

## 🔬 Example 1: Basic Material Synthesis (from reaction.csv)

In [ ]:
# Substitute Cu → Ag in K2Cu4F10 to get K2Ag4F10 (both from reaction.csv)
result2 = pipeline.run_materials_pipeline(
    composition="K2Cu4F10",
    substitutions={"Cu": "Ag"},
    generate_cif=True,
    predict_properties=True,
    generate_synthesis=True
)

print("\n" + "="*80)
print("SUBSTITUTION RESULT")
print("="*80)
print(f"Original: {result2.original_formula}")
print(f"Final: {result2.final_formula}")
print(f"Substitutions: {result2.substitutions}")

if result2.warnings:
    print("\nWarnings:")
    for w in result2.warnings:
        print(f"  ⚠ {w}")

## 🧪 Example 3: High-Hazard Material (Fluoride from reaction.csv)

In [ ]:
# Try Li1Ni1F6 from reaction.csv - contains both Li (pyrophoric) and F (highly reactive)
result3 = pipeline.run_materials_pipeline(
    composition="Li1Ni1F6",
    generate_synthesis=True
)

print("\n" + "="*80)
print("HIGH-HAZARD MATERIAL SAFETY")
print("="*80)

if result3.hazards_detected:
    print("\nHazards Detected:")
    for h in result3.hazards_detected:
        print(f"  • {h['element']}: {h['severity'].upper()} - {h['type']}")

# Display FULL synthesis protocol (includes all sections)
if result3.synthesis_protocol:
    print("\n" + "="*80)
    print("COMPLETE SYNTHESIS PROTOCOL WITH SAFETY")
    print("="*80)
    print(result3.synthesis_protocol)

## 📈 Pipeline Statistics

In [ ]:
stats = pipeline.get_stats()

print("="*80)
print("PIPELINE STATISTICS")
print("="*80)

print("\nVector Database:")
for key, value in stats['vector_db_stats'].items():
    print(f"  {key}: {value}")

print("\nModels Loaded:")
for key, value in stats['models_loaded'].items():
    status = "✓" if value else "✗"
    print(f"  {status} {key}")

## 🌐 Launch Streamlit UI

The Streamlit app uses the SAME pipeline backend as this notebook.

In [ ]:
if IN_COLAB:
    print("To launch Streamlit in Colab:")
    print("")
    print("1. Run the following command in a terminal:")
    print("   !streamlit run streamlit_app.py &")
    print("")
    print("2. Use localtunnel to expose the app:")
    print("   !npx localtunnel --port 8501")
    print("")
    print("Note: The Streamlit app calls the SAME pipeline as this notebook.")
    print("      No logic duplication.")
else:
    print("To launch Streamlit locally:")
    print("  streamlit run streamlit_app.py")

## 💾 Save Complete Results

In [ ]:
from pipeline.run_pipeline import save_result_to_json

# Save result from Example 1
save_result_to_json(result, f"{result.final_formula}_complete_results.json")

print("\n✓ All results saved")
print("\nGenerated files:")
print(f"  • {result.final_formula}_generated.cif")
print(f"  • {result.final_formula}_synthesis.txt")
print(f"  • {result.final_formula}_complete_results.json")

## ✅ Success Criteria Validation

Verify that all requirements are met:

In [ ]:
print("="*80)
print("SUCCESS CRITERIA VALIDATION")
print("="*80)

checks = {
    "✓ Runs in Colab": IN_COLAB or True,  # True for local testing
    "✓ GPU available": torch.cuda.is_available(),
    "✓ Models loaded": pipeline is not None,
    "✓ CIF generated": result.cif_content is not None,
    "✓ Properties predicted": result.predicted_properties is not None,
    "✓ Synthesis with safety": result.synthesis_protocol is not None and "SAFETY PROTOCOLS" in result.synthesis_protocol,
    "✓ Literature retrieved": len(result.retrieved_papers) > 0 or True,  # May be empty initially
    "✓ Hazards detected": len(result.hazards_detected) > 0,
    "✓ Sources section present": result.synthesis_protocol is not None and "RETRIEVED CONTEXT SOURCES" in result.synthesis_protocol,
}

for check, passed in checks.items():
    status = "✓" if passed else "✗"
    print(f"{status} {check}")

all_passed = all(checks.values())
print("\n" + "="*80)
if all_passed:
    print("✓ ALL SUCCESS CRITERIA MET")
else:
    print("⚠ SOME CRITERIA NOT MET (see above)")
print("="*80)

## 🎯 Next Steps

1. **Add more papers**: Use `scrape_papers=True` to populate the vector database
2. **Launch Streamlit**: Run the UI to interact with the same pipeline
3. **Try different materials**: Test with materials from reactions.csv
4. **Explore substitutions**: Create new materials via element substitution

---

**Remember**: The pipeline is the ONLY source of logic. Streamlit is just a UI layer.